In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import clear_output 
from IPython.display import Image
import os
from os import path
import pandas as pd
from scipy.signal import find_peaks
from scipy.stats import mannwhitneyu
import glob
import math
import time

In [2]:
def fits_data_to_3d_array(file_list): 
    '''
    Takes in a list of fits files and converts data into a 3d array
    Parameters:
        file_list (list): List containing paths of FITS files to convert.
    Returns:
        final_array: 3d array of data from each fits file.
    '''    
    for i, file in enumerate(file_list):
        with fits.open(file) as hdu:
            if i==0:  #If first iteration, create array with correct shape of data
                final_array = np.zeros((len(file_list), *hdu[0].data.shape))
            final_array[i,:,:] = hdu[0].data
    return final_array

#Creates a median combine of a 3d array
def median_combine(frame_array_3d):
    master_frame = np.median(frame_array_3d,axis=0) #takes median along the 'file_index' axis
    return master_frame
 
def save_array_to_fits_file(array, new_file_name):
    '''
    Saves a 2d array of pixel data to a FITS file.
    Parameters:
    array(2d): 2d array containing pixel values.
    new_file_name (path): location, name of FITS file to be saved.
    '''  
    hdu = fits.PrimaryHDU(data = array)
    hdul = fits.HDUList([hdu])
    hdul.writeto(new_file_name,overwrite=True)


def create_file_list(data_directory, file_name_to_glob, show_list = False):
    
    '''
    This creates a list of all FITS files that will be used.
    
    Parameters:
    data_directory : path to data folder
    file_name_to_glob : name of file containing data (ex.: spectrograph_flats_20s_)
                        This will glob together all files containing this keywords.
    show_list : Show file list for verification.
    '''
    file_list = glob.glob(os.path.join(data_directory, '*' + file_name_to_glob + '*'))
    
    if show_list: 
        for file in file_list: print(file)
            
    return file_list

def cut_pixel_data_array(pixel_data_array, y_pixel_min, y_pixel_max):
    '''
    Returns an array of pixel data, cut to a specified vertical (y-pixel) range.
    Parameters:
        pixel_data_array (2d array): Array of pixel data to create a cut from.
        y_pixel_min (int): Lower bound to start cut. 
        y_pixel_max (int): Upper bound to end cut. 
    Returns:
        final_array: 3d array of data from each fits file.
    '''   
    lower_index = (-1) + y_pixel_min
    upper_index = (-1) + y_pixel_max
    pixel_data_array_cut = pixel_data_array[lower_index:upper_index,:]
    print(pixel_data_array_cut.shape)
    return pixel_data_array_cut

def display_2d_array(pixel_data_array, lower_percentile=1, upper_percentile=99,figsize=(6,6),cmap='gray'):
    '''
    Displays a 2d array of pixel data. Useful to see the image contained in a FITS file,
    or to verify successful array data handling.
    Parameters:
        pixel_data_array (2d array): Array of pixel data to display.
        lower_percentile (int): Adjusts the pixel scale lower bound; default = 1
        upper_percentile (int): Adjusts the pixel scale upper bound; default = 99
    '''   
    minimum = np.percentile(pixel_data_array,lower_percentile)
    maximum = np.percentile(pixel_data_array,upper_percentile)
    plt.figure(figsize=figsize)
    plt.imshow(pixel_data_array, cmap=cmap, origin = 'lower', vmin=minimum, vmax=maximum)
    plt.colorbar(location='top')
    
def glob_files(folder, specifier, asterisks='both', show=False):
    asterisk = "*"
    if asterisks == 'both':
        files = glob.glob(os.path.join(folder, asterisk + specifier + asterisk))
    elif asterisks == 'left':
        files = glob.glob(os.path.join(folder, asterisk + specifier))
    elif asterisks == 'right':
        files = glob.glob(os.path.join(folder, specifier + asterisk))
    if show:
        print(f'Number of Files: {len(files)}')
        for file in files: 
            print(file)
    return files

In [3]:
def fits_data_to_3d_array(file_list): 
    for i, file in enumerate(file_list):
        with fits.open(file) as hdu:
            if i==0:  #If first iteration, create array with correct shape of data
                final_array = np.zeros((len(file_list), *hdu[0].data.shape))
            final_array[i,:,:] = hdu[0].data
    return final_array

def median_combine(frame_array_3d):
    master_frame = np.median(frame_array_3d,axis=0) 
    return master_frame

def mean_combine(frame_array_3d):
    master_frame = np.mean(frame_array_3d,axis=0) #takes median along the 'file_index' axis
    return master_frame
    
def save_array_as_fits(array, new_file_name):
    hdu = fits.PrimaryHDU(data = array)
    hdul = fits.HDUList([hdu])
    hdul.writeto(new_file_name)

def cut_images_array(image_array, y_pixel_min, y_pixel_max):
    lower_index = (-1) + y_pixel_min
    upper_index = (-1) + y_pixel_max
    image_array_cut = image_array[lower_index:upper_index,:]
    print(image_array_cut.shape)
    return image_array_cut

def display_2d_array(image_array, lower_percentile=1, upper_percentile=99):
    minimum = np.percentile(image_array,lower_percentile)
    maximum = np.percentile(image_array,upper_percentile)
    plt.imshow(image_array, cmap='gray', origin = 'lower', vmin=minimum, vmax=maximum)
    plt.colorbar(location='top')

In [4]:
#note: the pretty picture file directly with the fits files is assumed to be in the same directory as the notebook.
#also, the note that there's a space, not underscores, in the name. This was why I had problems running the pretty picture code.

long_darks_3d_array = fits_data_to_3d_array(glob_files("pretty picture","orion_dark_18s"))
short_darks_3d_array = fits_data_to_3d_array(glob_files("pretty picture","orion_dark_.5s"))
flats_3d_array = fits_data_to_3d_array(glob_files("pretty picture","orion_blue_flat"))

masterdark_long = median_combine(long_darks_3d_array)
masterdark_short = median_combine(short_darks_3d_array)
masterflat = median_combine(flats_3d_array)
masterflat_normalized = (masterflat - masterdark_short) / np.median(masterflat - masterdark_short)

In [5]:
sciences_3d_array = fits_data_to_3d_array(glob_files("Lab 2","HATP6_blue_lights_18s"))

sciences_3d_array_calibrated = sciences_3d_array.copy()
num_science_files = len(sciences_3d_array_calibrated[:,0,0])
for i in range(num_science_files):
    sciences_3d_array_calibrated[i,:,:] = (sciences_3d_array_calibrated[i,:,:] - masterdark_long) / masterflat_normalized
    save_array_to_fits_file(sciences_3d_array_calibrated[i,:,:], f"new_lab2_calibration/new_science_calib_{i}.fits")

#make a folder calles "new_lab2_calibration" in the same directory as this notebook and save the new calibration files to it